# **Soccer Data Analysis Using SQL**

# Notebook Purpose

This notebook I used to practice some SQL techniques from open-source Kaggle data(Exploring the European Soccer Database with SQL). We will explore the European Soccer DataBase to gain the insight from the dataset with SQL techniques.

**The European Soccer Database**

The European Soccer Database contains information on the teams, match results and players for the top division football leagues across Europe. We can look at every season between 08/09 and 15/16.


**Import and Connect**


The below code block sets up the pandas and sqlite3 libraries, so we can use this python based notebook to run SQL queries, as the European Soccer Database comes in the sqlite format.


In [1]:
#Imports

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt

path = "../input/soccer"  #Insert path here
database = path + 'database.sqlite'


conn = sqlite3.connect('../input/soccer/database.sqlite')

Read an SQL table (Since this time we run the code in the python, we need to surround SQL with python)

In [2]:
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


From the above, there are 7 tables representing many aspects of the europen soccer

**Summarize information in the case of Spain Country**

Summarize Soccer from different database. We will use a match database as a main database to summarize data using LEFT JOIN technique(to preserve information in a main database) to extract data from other tables.

In [4]:
detailed_matches = pd.read_sql("""
SELECT Match.id, 
Country.name AS country_name, 
League.name AS league_name, 
season, 
date,
HT.team_long_name AS  home_team,
AT.team_long_name AS away_team,
home_team_goal, 
away_team_goal                                        
FROM Match
LEFT JOIN Country on Country.id = Match.country_id
LEFT JOIN League on League.id = Match.league_id
LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
WHERE country_name = 'Spain'
AND season = '2015/2016'
ORDER by date
LIMIT 10;""", conn)
detailed_matches

,id,country_name,league_name,season,date,home_team,away_team,home_team_goal,away_team_goal
0,24180,Spain,Spain LIGA BBVA,2015/2016,2015-08-21 00:00:00,Málaga CF,Sevilla FC,0,0
1,24179,Spain,Spain LIGA BBVA,2015/2016,2015-08-22 00:00:00,Atlético Madrid,UD Las Palmas,1,0
2,24183,Spain,Spain LIGA BBVA,2015/2016,2015-08-22 00:00:00,RCD Espanyol,Getafe CF,1,0
3,24186,Spain,Spain LIGA BBVA,2015/2016,2015-08-22 00:00:00,RC Deportivo de La Coruña,Real Sociedad,0,0
4,24187,Spain,Spain LIGA BBVA,2015/2016,2015-08-22 00:00:00,Rayo Vallecano,Valencia CF,0,0
5,24178,Spain,Spain LIGA BBVA,2015/2016,2015-08-23 00:00:00,Levante UD,RC Celta de Vigo,1,2
6,24181,Spain,Spain LIGA BBVA,2015/2016,2015-08-23 00:00:00,Athletic Club de Bilbao,FC Barcelona,0,1
7,24184,Spain,Spain LIGA BBVA,2015/2016,2015-08-23 00:00:00,Real Betis Balompié,Villarreal CF,1,1
8,24185,Spain,Spain LIGA BBVA,2015/2016,2015-08-23 00:00:00,Real Sporting de Gijón,Real Madrid CF,0,0
9,24182,Spain,Spain LIGA BBVA,2015/2016,2015-08-24 00:00:00,Granada CF,SD Eibar,1,3


**Focus on the Tottenham Hotspur F.C. match**

Personally, I am a big fan of Chelsea Football club. Hence, this time I will apply SQL techniques to focus the enemy of Chelsea Football Club(Tottenham Hotspur F.C.)

In [5]:
tot = pd.read_sql("""
SELECT m.date,
HT.team_long_name AS  home_team,
AT.team_long_name AS away_team,
home_team_goal, 
away_team_goal, 
CASE WHEN m.home_team_api_id = 8586 and m.home_team_goal > m.away_team_goal THEN 'Spurs won'
WHEN m.home_team_api_id = 8586 and  m.home_team_goal < m.away_team_goal THEN 'Spurs lost'
WHEN m.away_team_api_id = 8586 and m.home_team_goal < m.away_team_goal THEN 'Spurs won'
WHEN m.away_team_api_id = 8586 and m.home_team_goal > m.away_team_goal THEN 'Spurs lost'
ELSE 'Tie' END AS outcome
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
Order by 1;""", conn)
tot

,date,home_team,away_team,home_team_goal,away_team_goal,outcome
0,2008-08-16 00:00:00,Middlesbrough,Tottenham Hotspur,2,1,Spurs lost
1,2008-08-23 00:00:00,Tottenham Hotspur,Sunderland,1,2,Spurs lost
2,2008-08-31 00:00:00,Chelsea,Tottenham Hotspur,1,1,Tie
3,2008-09-15 00:00:00,Tottenham Hotspur,Aston Villa,1,2,Spurs lost
4,2008-09-21 00:00:00,Tottenham Hotspur,Wigan Athletic,0,0,Tie
...,...,...,...,...,...,...
299,2016-04-18 00:00:00,Stoke City,Tottenham Hotspur,0,4,Spurs won
300,2016-04-25 00:00:00,Tottenham Hotspur,West Bromwich Albion,1,1,Tie
301,2016-05-02 00:00:00,Chelsea,Tottenham Hotspur,2,2,Tie
302,2016-05-08 00:00:00,Tottenham Hotspur,Southampton,1,2,Spurs lost


Summarize the performance of Spurs by season and by outcome(respectively). Hence, we will apply GroupBy method to summarize.

In [6]:
totout = pd.read_sql("""
SELECT
m.season,
CASE WHEN m.home_team_api_id = 8586 and m.home_team_goal > m.away_team_goal THEN 'Spurs won'
WHEN m.home_team_api_id = 8586 and  m.home_team_goal < m.away_team_goal THEN 'Spurs lost'
WHEN m.away_team_api_id = 8586 and m.home_team_goal < m.away_team_goal THEN 'Spurs won'
WHEN m.away_team_api_id = 8586 and m.home_team_goal > m.away_team_goal THEN 'Spurs lost'
ELSE 'Tie' END AS outcome,
count (m.date) AS count
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
GROUP BY m.season,outcome;""", conn)
totout

,season,outcome,count
0,2008/2009,Spurs lost,15
1,2008/2009,Spurs won,14
2,2008/2009,Tie,9
3,2009/2010,Spurs lost,10
4,2009/2010,Spurs won,21
5,2009/2010,Tie,7
6,2010/2011,Spurs lost,8
7,2010/2011,Spurs won,16
8,2010/2011,Tie,14
9,2011/2012,Spurs lost,9


Summarize the total goal in each season.

In [7]:
GoalsBySeason = pd.read_sql("""
SELECT
m.season,
SUM(CASE WHEN m.home_team_api_id = 8586  THEN home_team_goal END) AS home_goals,
SUM(CASE WHEN m.away_team_api_id = 8586  THEN away_team_goal END) AS away_goals
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
GROUP BY m.season;""", conn)
GoalsBySeason

,season,home_goals,away_goals
0,2008/2009,21,24
1,2009/2010,40,27
2,2010/2011,30,25
3,2011/2012,39,27
4,2012/2013,29,37
5,2013/2014,30,25
6,2014/2015,31,27
7,2015/2016,35,34


Get Average home goal and Away goal

In [8]:
AvgGoalsBySeason = pd.read_sql("""
SELECT
m.season,
ROUND(AVG(CASE WHEN m.home_team_api_id = 8586  THEN home_team_goal END),2) AS average_home_goals,
ROUND(AVG(CASE WHEN m.home_team_api_id = 8586  THEN away_team_goal END),2) AS average_away_goals
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
GROUP BY m.season;""", conn)
AvgGoalsBySeason

,season,average_home_goals,average_away_goals
0,2008/2009,1.11,0.53
1,2009/2010,2.11,0.63
2,2010/2011,1.58,1.00
3,2011/2012,2.05,0.89
4,2012/2013,1.53,0.95
5,2013/2014,1.58,1.21
6,2014/2015,1.63,1.26
7,2015/2016,1.84,0.79


**Comparing different result across different leagues**

1) Finding what percentage of mathces home team wining the match

2) Finding what percentage of matches end up in a draw

3) Finding the average goal from each league

The three fields (country, percentage draws 2013/14, percentage draws 2014/15) are grouped by country.


In [14]:
homewin = pd.read_sql("""
SELECT c.name AS country,
ROUND(AVG(CASE WHEN m.season='2013/2014' AND m.home_team_goal > m.away_team_goal THEN 1
 WHEN m.season='2013/2014' AND m.home_team_goal != m.away_team_goal THEN 0
 WHEN m.season='2013/2014' AND m.home_team_goal < m.away_team_goal THEN 0
END),2) AS pct_homewin_2013_2014,
ROUND(AVG(CASE WHEN m.season='2014/2015' AND m.home_team_goal > m.away_team_goal THEN 1
 WHEN m.season='2014/2015' AND m.home_team_goal != m.away_team_goal THEN 0
 WHEN m.season='2014/2015' AND m.home_team_goal < m.away_team_goal THEN 0
END),2) AS pct_homewin_2014_2015
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
GROUP BY country
;""", conn)
homewin

,country,pct_homewin_2013_2014,pct_homewin_2014_2015
0,Belgium,0.60,0.59
1,England,0.59,0.60
2,France,0.62,0.62
3,Germany,0.60,0.65
4,Italy,0.62,0.59
5,Netherlands,0.65,0.59
6,Poland,0.66,0.66
7,Portugal,0.60,0.62
8,Scotland,0.57,0.55
9,Spain,0.61,0.59


In [15]:
perdraw = pd.read_sql("""
SELECT c.name AS country,
ROUND(AVG(CASE WHEN m.season='2013/2014' AND m.home_team_goal = m.away_team_goal THEN 1
 WHEN m.season='2013/2014' AND m.home_team_goal != m.away_team_goal THEN 0
END),2) AS pct_ties_2013_2014,
ROUND(AVG(CASE WHEN m.season='2014/2015' AND m.home_team_goal = m.away_team_goal THEN 1
 WHEN m.season='2014/2015' AND m.home_team_goal != m.away_team_goal THEN 0
END),2) AS pct_ties_2014_2015
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
GROUP BY country
;""", conn)
perdraw

,country,pct_ties_2013_2014,pct_ties_2014_2015
0,Belgium,0.17,0.25
1,England,0.21,0.24
2,France,0.28,0.23
3,Germany,0.21,0.27
4,Italy,0.24,0.32
5,Netherlands,0.27,0.24
6,Poland,0.30,0.28
7,Portugal,0.25,0.28
8,Scotland,0.22,0.19
9,Spain,0.23,0.24


In the 2014/2015 season, Italy had the highest proportion of draws (almost 1/3) and Scotland had the lowest (about 1/5). 

In [16]:
leagues_by_avg_goal = pd.read_sql("""
SELECT 
l.name AS league,
ROUND(AVG(m.home_team_goal + m.away_team_goal),2) AS avg_goals
FROM league as L
LEFT JOIN match AS m
ON l.id = m.country_id
WHERE m.season = '2014/2015'
GROUP BY league
;""", conn)
leagues_by_avg_goal

,league,avg_goals
0,Belgium Jupiler League,2.78
1,England Premier League,2.57
2,France Ligue 1,2.49
3,Germany 1. Bundesliga,2.75
4,Italy Serie A,2.69
5,Netherlands Eredivisie,3.08
6,Poland Ekstraklasa,2.62
7,Portugal Liga ZON Sagres,2.49
8,Scotland Premier League,2.57
9,Spain LIGA BBVA,2.66


We can see that the Netherlands had the highest average number of goals with 3.08. France and Portugal both had roughly the lowest number of average goals with 2.49.

The script below I try to practice subquery.

Let's start by finding the average total goal in each match across the entired dataset.

In [18]:
avg_goal1415 = pd.read_sql("""
SELECT 
ROUND(AVG(m.home_team_goal + m.away_team_goal),2) AS avg_goals
FROM match AS m
WHERE m.season = '2014/2015';""", conn)
avg_goal1415

,avg_goals
0,2.68


Based on the average score, we will the match which the team has the total sum goal more than entire average goal


In [20]:
Higher_than_avg_goals1415 = pd.read_sql("""
SELECT 
HT.team_long_name AS  home_team,
AT.team_long_name AS away_team,
m.home_team_goal,
m.away_team_goal
FROM match AS m
LEFT JOIN League as l
ON l.id = m.country_id
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.season = '2014/2015'
AND l.name = 'France Ligue 1'
AND (m.home_team_goal + m.away_team_goal) >
(SELECT 
    AVG(m.home_team_goal + m.away_team_goal) AS avg_goals
    FROM match AS m
WHERE m.season = '2014/2015');""", conn)
Higher_than_avg_goals1415

,home_team,away_team,home_team_goal,away_team_goal
0,Stade de Reims,Paris Saint-Germain,2,2
1,SC Bastia,Olympique de Marseille,3,3
2,Évian Thonon Gaillard FC,SM Caen,0,3
3,AS Monaco,FC Lorient,1,2
4,OGC Nice,Toulouse FC,3,2
...,...,...,...,...
165,FC Metz,Stade de Reims,3,0
166,Montpellier Hérault SC,En Avant de Guingamp,2,1
167,SM Caen,Olympique de Marseille,1,2
168,Évian Thonon Gaillard FC,FC Metz,3,0


Select the match more than 10 goals 

In [21]:
matchesabove10 = pd.read_sql("""
SELECT 
country,date,total_goals
FROM 
(SELECT 
c.name as country,
m.date,
m.home_team_goal,
m.away_team_goal,
(m.home_team_goal + m.away_team_goal) as total_goals
from match AS m
left join country as c
ON m.country_id = c.id) AS subquery
WHERE 
total_goals >=10
    ;""", conn)
matchesabove10

,country,date,total_goals
0,England,2009-11-22 00:00:00,10
1,England,2011-08-28 00:00:00,10
2,England,2012-12-29 00:00:00,10
3,England,2013-05-19 00:00:00,10
4,France,2009-11-08 00:00:00,10
5,Germany,2013-03-30 00:00:00,11
6,Netherlands,2010-10-24 00:00:00,10
7,Netherlands,2011-11-06 00:00:00,10
8,Scotland,2010-05-05 00:00:00,12
9,Spain,2013-10-30 00:00:00,10


wow interesting. There are 13 match scoring than 10 goals.

Average and its difference from overall league

In [23]:
goalsdiff = pd.read_sql("""
Select 
l.name as league,
round((avg(m.home_team_goal + m.away_team_goal)),2) as avg_goals,
round(avg(m.home_team_goal + m.away_team_goal)
-((Select avg(home_team_goal + away_team_goal) from match WHERE season = '2014/2015')),2) as diff 
FROM league as l
LEFT JOIN match as m
ON l.country_id = m.country_id
WHERE m.season = '2014/2015'
Group by l.name
    ;""", conn)
goalsdiff

,league,avg_goals,diff
0,Belgium Jupiler League,2.78,0.11
1,England Premier League,2.57,-0.11
2,France Ligue 1,2.49,-0.18
3,Germany 1. Bundesliga,2.75,0.08
4,Italy Serie A,2.69,0.01
5,Netherlands Eredivisie,3.08,0.40
6,Poland Ekstraklasa,2.62,-0.06
7,Portugal Liga ZON Sagres,2.49,-0.18
8,Scotland Premier League,2.57,-0.10
9,Spain LIGA BBVA,2.66,-0.02


Overall, we have leverage SQL technique to find the insight from the soccer dataset.